In [1]:
import json
import os
from glob import glob
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import wandb

from IPython.display import display, HTML

In [2]:
api = wandb.Api()

In [3]:
# The W&B API also provides a way for you to query across runs in a project with api.runs(). The most common use case is exporting runs data for custom analysis.  The query interface is the same as the one MongoDB uses.
# wandb.init("analysis", mode="offline")
def download_test_runs(overwrite=False):
    runs = api.runs("ajayj/nerf-nl", {"$and": [{"config.render_only": True, "config.render_test": True, "state": "finished"}]},
                    order='-created_at')
    print("Found %i" % len(runs))

    # Get most recent run of each
    runs_by_expname = {}
    for run in runs:
        expname = run.config["expname"]
        if expname not in runs_by_expname:
            runs_by_expname[expname] = run
    print("Filtered to %i most recent and unique" % len(runs_by_expname))

    # Download metrics
    for run in runs_by_expname.values():
        #metrics_path = f'logs/{run.config["expname"]}/renderonly_test_*/test_metrics.json'
        files = run.files()
        for file in files:
            if file.name.startswith(f'logs/{run.config["expname"]}/renderonly_test_') and file.name.endswith('/test_metrics.json'):
                if not overwrite and os.path.exists(file.name):
                    continue
                print('Downloading', file.name)

                try:
                    file.download(root='.', replace=True)
                except Exception as e:
                    print('download error', file.name, e)

    return runs_by_expname

In [4]:
runs_by_expname = download_test_runs()

Found 44
Filtered to 27 most recent and unique


In [5]:
re.match?

Signature: re.match(pattern, string, flags=0)
Docstring:
Try to apply the pattern at the start of the string, returning
a Match object, or None if no match was found.
File:      ~/miniconda3/envs/clip/lib/python3.8/re.py
Type:      function


In [6]:
# Load metrics into a dataframe
print("Skipping", set(glob('logs/*/renderonly_test_*/test_metrics.json')) - set(glob('logs/*/renderonly_test_199999/test_metrics.json')))

metrics_list = []
for metrics_path in glob('logs/*/renderonly_test_*/test_metrics.json'):
    with open(metrics_path, 'r') as metrics_f:
        metrics_and_args = json.load(metrics_f)
        
        # extract iteration count
        # metrics_and_args['test_metrics_path'] = metrics_path
        m = re.match(r'logs/.+/renderonly_test_([0-9]+)/test_metrics.json', metrics_path)
        assert m
        metrics_and_args['train_iters'] = int(m.group(1))
        # extract training view count
        metrics_and_args['views'] = metrics_and_args["args"]['max_train_views']
        metrics_and_args['views'] = metrics_and_args['views'] if metrics_and_args['views'] >= 0 else 100

        # add columns for arguments
        for k, v in metrics_and_args["args"].items():
            metrics_and_args[f"args.{k}"] = v
        del metrics_and_args["args"]

        metrics_list.append(metrics_and_args)

metrics_df = pd.DataFrame(metrics_list)

Skipping {'logs/240_blender_chair_4views_ctr/renderonly_test_119999/test_metrics.json', 'logs/244_blender_mic_4views_ctr/renderonly_test_109999/test_metrics.json', 'logs/245_blender_ship_4views_ctr/renderonly_test_109999/test_metrics.json', 'logs/241_blender_drums_4views_ctr/renderonly_test_119999/test_metrics.json', 'logs/242_blender_ficus_4views_ctr/renderonly_test_119999/test_metrics.json', 'logs/251_blender_drums_8views_ctr/renderonly_test_039999/test_metrics.json', 'logs/251_blender_drums_8views_ctr/renderonly_test_029999/test_metrics.json', 'logs/250_blender_chair_8views_ctr/renderonly_test_029999/test_metrics.json', 'logs/250_blender_chair_8views_ctr/renderonly_test_039999/test_metrics.json', 'logs/243_blender_lego_4views_ctr/renderonly_test_109999/test_metrics.json'}


In [7]:
for scene_key, scene_df in metrics_df.groupby('args.datadir'):
    print(scene_key)
    scene_df = scene_df.sort_values([
        'views',
        'train_iters'
    ])
    display(scene_df)
#     print('==='*20)

./data/nerf_synthetic/chair


,mse,psnr,ssim,lpips,train_iters,views,args.config,args.expname,args.basedir,args.datadir,...,args.patch_gan_lr,args.patch_gan_beta1,args.patch_gan_num_Ds,args.patch_gan_netD,args.patch_gan_ndf,args.patch_gan_norm,args.patch_gan_init_type,args.patch_gan_init_gain,args.patch_gan_nl,args.n_gpus
14,0.018246,18.315204,0.798149,0.197383,119999,4,configs/240_blender_chair_4views_ctr.txt,240_blender_chair_4views_ctr,./logs,./data/nerf_synthetic/chair,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
13,0.052804,12.973991,0.601996,0.415411,199999,4,configs/220_blender_chair_4views.txt,220_blender_chair_4views,./logs,./data/nerf_synthetic/chair,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
27,0.004263,24.179489,0.881425,0.117759,29999,8,configs/250_blender_chair_8views_ctr.txt,250_blender_chair_8views_ctr,./logs,./data/nerf_synthetic/chair,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
28,0.003754,24.611861,0.888030,0.112686,39999,8,configs/250_blender_chair_8views_ctr.txt,250_blender_chair_8views_ctr,./logs,./data/nerf_synthetic/chair,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
12,0.011285,21.048566,0.860601,0.162509,199999,8,configs/230_blender_chair_8views.txt,230_blender_chair_8views,./logs,./data/nerf_synthetic/chair,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
9,0.000412,34.073013,0.977810,0.024821,199999,100,configs/chair.txt,blender_paper_chair,./logs,./data/nerf_synthetic/chair,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1


./data/nerf_synthetic/drums


,mse,psnr,ssim,lpips,train_iters,views,args.config,args.expname,args.basedir,args.datadir,...,args.patch_gan_lr,args.patch_gan_beta1,args.patch_gan_num_Ds,args.patch_gan_netD,args.patch_gan_ndf,args.patch_gan_norm,args.patch_gan_init_type,args.patch_gan_init_gain,args.patch_gan_nl,args.n_gpus
11,0.045789,13.444961,0.673204,0.308898,119999,4,configs/241_blender_drums_4views_ctr.txt,241_blender_drums_4views_ctr,./logs,./data/nerf_synthetic/drums,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
21,0.086626,10.671803,0.590800,0.423424,199999,4,configs/221_blender_drums_4views.txt,221_blender_drums_4views,./logs,./data/nerf_synthetic/drums,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
16,0.009560,20.299326,0.839417,0.138345,29999,8,configs/251_blender_drums_8views_ctr.txt,251_blender_drums_8views_ctr,./logs,./data/nerf_synthetic/drums,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
17,0.009499,20.333689,0.843306,0.129298,39999,8,configs/251_blender_drums_8views_ctr.txt,251_blender_drums_8views_ctr,./logs,./data/nerf_synthetic/drums,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
10,0.018335,17.471725,0.769530,0.231024,199999,8,configs/231_blender_drums_8views.txt,231_blender_drums_8views,./logs,./data/nerf_synthetic/drums,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
18,0.002971,25.529593,0.928950,0.063527,199999,100,configs/drums.txt,blender_paper_drums,./logs,./data/nerf_synthetic/drums,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1


./data/nerf_synthetic/ficus


...

,mse,psnr,ssim,lpips,train_iters,views,args.config,args.expname,args.basedir,args.datadir,...,args.patch_gan_lr,args.patch_gan_beta1,args.patch_gan_num_Ds,args.patch_gan_netD,args.patch_gan_ndf,args.patch_gan_norm,args.patch_gan_init_type,args.patch_gan_init_gain,args.patch_gan_nl,args.n_gpus
26,0.070833,11.619872,0.621727,0.362456,109999,4,configs/243_blender_lego_4views_ctr.txt,243_blender_lego_4views_ctr,./logs,./data/nerf_synthetic/lego,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
7,0.132863,8.943158,0.491976,0.475585,199999,4,configs/223_blender_lego_4views.txt,223_blender_lego_4views,./logs,./data/nerf_synthetic/lego,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
0,0.110414,9.726092,0.525642,0.466816,199999,8,configs/233_blender_lego_8views.txt,233_blender_lego_8views,./logs,./data/nerf_synthetic/lego,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
15,0.005002,23.896500,0.862688,0.110295,199999,8,configs/042_blender_paper_lego.txt,042_blender_paper_lego_ctr_coarseandfine_clip_...,./logs,./data/nerf_synthetic/lego,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
24,0.000721,31.617750,0.965375,0.032724,199999,100,configs/lego.txt,blender_paper_lego,./logs,./data/nerf_synthetic/lego,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1


./data/nerf_synthetic/mic


,mse,psnr,ssim,lpips,train_iters,views,args.config,args.expname,args.basedir,args.datadir,...,args.patch_gan_lr,args.patch_gan_beta1,args.patch_gan_num_Ds,args.patch_gan_netD,args.patch_gan_ndf,args.patch_gan_norm,args.patch_gan_init_type,args.patch_gan_init_gain,args.patch_gan_nl,args.n_gpus
8,0.012087,19.412153,0.877759,0.129028,109999,4,configs/244_blender_mic_4views_ctr.txt,244_blender_mic_4views_ctr,./logs,./data/nerf_synthetic/mic,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
23,0.058747,12.372348,0.707625,0.346917,199999,4,configs/224_blender_mic_4views.txt,224_blender_mic_4views,./logs,./data/nerf_synthetic/mic,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
2,0.003241,26.287436,0.944459,0.067203,199999,8,configs/234_blender_mic_8views.txt,234_blender_mic_8views,./logs,./data/nerf_synthetic/mic,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
3,0.000494,33.196878,0.978615,0.022581,199999,100,configs/mic.txt,blender_paper_mic,./logs,./data/nerf_synthetic/mic,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1


./data/nerf_synthetic/ship


,mse,psnr,ssim,lpips,train_iters,views,args.config,args.expname,args.basedir,args.datadir,...,args.patch_gan_lr,args.patch_gan_beta1,args.patch_gan_num_Ds,args.patch_gan_netD,args.patch_gan_ndf,args.patch_gan_norm,args.patch_gan_init_type,args.patch_gan_init_gain,args.patch_gan_nl,args.n_gpus
22,0.016845,17.917937,0.680254,0.296329,109999,4,configs/245_blender_ship_4views_ctr.txt,245_blender_ship_4views_ctr,./logs,./data/nerf_synthetic/ship,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
5,0.241816,6.332760,0.335781,0.593170,199999,4,configs/225_blender_ship_4views.txt,225_blender_ship_4views,./logs,./data/nerf_synthetic/ship,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
20,0.097770,12.929134,0.605394,0.375103,199999,8,configs/235_blender_ship_8views.txt,235_blender_ship_8views,./logs,./data/nerf_synthetic/ship,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
4,0.001215,29.407472,0.874514,0.124558,199999,100,configs/ship.txt,blender_paper_ship,./logs,./data/nerf_synthetic/ship,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1


In [8]:
# Select evaluations of most recent training iteration
idx = metrics_df.groupby('args.config')['train_iters'].transform(max) == metrics_df['train_iters']
metrics_df_max_iters = metrics_df[idx]

# Aggregate
grouped_df = metrics_df_max_iters\
    .sort_values('args.max_train_views')\
    .groupby(['views', 'args.consistency_loss'])

for k, v in grouped_df:
    print(k)
    mse = v['mse'].mean()
    psnr = v['psnr'].mean()
    ssim = v['ssim'].mean()
    lpips = v['lpips'].mean()
    display(v)
    print(f'{k[0]} views, mse={mse:.4f} psnr={psnr:.4f} ssim={ssim:.4f} lpips={lpips:.4f}')

(4, 'consistent_with_target_rep')


,mse,psnr,ssim,lpips,train_iters,views,args.config,args.expname,args.basedir,args.datadir,...,args.patch_gan_lr,args.patch_gan_beta1,args.patch_gan_num_Ds,args.patch_gan_netD,args.patch_gan_ndf,args.patch_gan_norm,args.patch_gan_init_type,args.patch_gan_init_gain,args.patch_gan_nl,args.n_gpus
22,0.016845,17.917937,0.680254,0.296329,109999,4,configs/245_blender_ship_4views_ctr.txt,245_blender_ship_4views_ctr,./logs,./data/nerf_synthetic/ship,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
14,0.018246,18.315204,0.798149,0.197383,119999,4,configs/240_blender_chair_4views_ctr.txt,240_blender_chair_4views_ctr,./logs,./data/nerf_synthetic/chair,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
26,0.070833,11.619872,0.621727,0.362456,109999,4,configs/243_blender_lego_4views_ctr.txt,243_blender_lego_4views_ctr,./logs,./data/nerf_synthetic/lego,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
8,0.012087,19.412153,0.877759,0.129028,109999,4,configs/244_blender_mic_4views_ctr.txt,244_blender_mic_4views_ctr,./logs,./data/nerf_synthetic/mic,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
25,0.077863,11.267622,0.621853,0.341617,119999,4,configs/242_blender_ficus_4views_ctr.txt,242_blender_ficus_4views_ctr,./logs,./data/nerf_synthetic/ficus,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
11,0.045789,13.444961,0.673204,0.308898,119999,4,configs/241_blender_drums_4views_ctr.txt,241_blender_drums_4views_ctr,./logs,./data/nerf_synthetic/drums,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1


4 views, mse=0.0403 psnr=15.3296 ssim=0.7122 lpips=0.2726
(4, 'none')


,mse,psnr,ssim,lpips,train_iters,views,args.config,args.expname,args.basedir,args.datadir,...,args.patch_gan_lr,args.patch_gan_beta1,args.patch_gan_num_Ds,args.patch_gan_netD,args.patch_gan_ndf,args.patch_gan_norm,args.patch_gan_init_type,args.patch_gan_init_gain,args.patch_gan_nl,args.n_gpus
21,0.086626,10.671803,0.590800,0.423424,199999,4,configs/221_blender_drums_4views.txt,221_blender_drums_4views,./logs,./data/nerf_synthetic/drums,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
23,0.058747,12.372348,0.707625,0.346917,199999,4,configs/224_blender_mic_4views.txt,224_blender_mic_4views,./logs,./data/nerf_synthetic/mic,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
13,0.052804,12.973991,0.601996,0.415411,199999,4,configs/220_blender_chair_4views.txt,220_blender_chair_4views,./logs,./data/nerf_synthetic/chair,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
7,0.132863,8.943158,0.491976,0.475585,199999,4,configs/223_blender_lego_4views.txt,223_blender_lego_4views,./logs,./data/nerf_synthetic/lego,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
6,0.018838,17.700611,0.827852,0.162645,199999,4,configs/222_blender_ficus_4views.txt,222_blender_ficus_4views,./logs,./data/nerf_synthetic/ficus,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
5,0.241816,6.332760,0.335781,0.593170,199999,4,configs/225_blender_ship_4views.txt,225_blender_ship_4views,./logs,./data/nerf_synthetic/ship,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1


4 views, mse=0.0986 psnr=11.4991 ssim=0.5927 lpips=0.4029
(8, 'consistent_with_target_rep')


...

8 views, mse=0.0061 psnr=22.9473 ssim=0.8647 lpips=0.1174
(8, 'none')


,mse,psnr,ssim,lpips,train_iters,views,args.config,args.expname,args.basedir,args.datadir,...,args.patch_gan_lr,args.patch_gan_beta1,args.patch_gan_num_Ds,args.patch_gan_netD,args.patch_gan_ndf,args.patch_gan_norm,args.patch_gan_init_type,args.patch_gan_init_gain,args.patch_gan_nl,args.n_gpus
20,0.097770,12.929134,0.605394,0.375103,199999,8,configs/235_blender_ship_8views.txt,235_blender_ship_8views,./logs,./data/nerf_synthetic/ship,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
0,0.110414,9.726092,0.525642,0.466816,199999,8,configs/233_blender_lego_8views.txt,233_blender_lego_8views,./logs,./data/nerf_synthetic/lego,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
12,0.011285,21.048566,0.860601,0.162509,199999,8,configs/230_blender_chair_8views.txt,230_blender_chair_8views,./logs,./data/nerf_synthetic/chair,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
10,0.018335,17.471725,0.769530,0.231024,199999,8,configs/231_blender_drums_8views.txt,231_blender_drums_8views,./logs,./data/nerf_synthetic/drums,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
2,0.003241,26.287436,0.944459,0.067203,199999,8,configs/234_blender_mic_8views.txt,234_blender_mic_8views,./logs,./data/nerf_synthetic/mic,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
19,0.043530,13.728358,0.660588,0.353633,199999,8,configs/232_blender_ficus_8views.txt,232_blender_ficus_8views,./logs,./data/nerf_synthetic/ficus,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1


8 views, mse=0.0474 psnr=16.8652 ssim=0.7277 lpips=0.2760
(100, 'none')


,mse,psnr,ssim,lpips,train_iters,views,args.config,args.expname,args.basedir,args.datadir,...,args.patch_gan_lr,args.patch_gan_beta1,args.patch_gan_num_Ds,args.patch_gan_netD,args.patch_gan_ndf,args.patch_gan_norm,args.patch_gan_init_type,args.patch_gan_init_gain,args.patch_gan_nl,args.n_gpus
9,0.000412,34.073013,0.977810,0.024821,199999,100,configs/chair.txt,blender_paper_chair,./logs,./data/nerf_synthetic/chair,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
1,0.001242,29.162503,0.966199,0.035262,199999,100,configs/ficus.txt,blender_paper_ficus,./logs,./data/nerf_synthetic/ficus,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
3,0.000494,33.196878,0.978615,0.022581,199999,100,configs/mic.txt,blender_paper_mic,./logs,./data/nerf_synthetic/mic,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
4,0.001215,29.407472,0.874514,0.124558,199999,100,configs/ship.txt,blender_paper_ship,./logs,./data/nerf_synthetic/ship,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
18,0.002971,25.529593,0.928950,0.063527,199999,100,configs/drums.txt,blender_paper_drums,./logs,./data/nerf_synthetic/drums,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1
24,0.000721,31.617750,0.965375,0.032724,199999,100,configs/lego.txt,blender_paper_lego,./logs,./data/nerf_synthetic/lego,...,0.0002,0.5,1,basic_256_multi,64,instance,xavier,0.02,relu,1


100 views, mse=0.0012 psnr=30.4979 ssim=0.9486 lpips=0.0506


In [9]:
!ls /data/ajay/clip/nerf/nerf-pytorch/data/nerf_synthetic/chair/train -1 | wc -l

100
